# Modelo de Ia (Gaussian)

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import  train_test_split,KFold,cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer,make_column_selector

In [2]:
df = pd.read_excel('../base/Incluses.xlsx')

df = df.drop(columns=['Id', 'Hora de início','Hora de conclusão','Email','Nome','Coluna1'])

display(df.head(5))

,Você participa da comunidade LGBTQIA+,Idade,Gênero,Orientação Sexual,Localização,Nível de Escolaridade,Você usa aplicativos para procurar oportunidades de emprego?,Preferência por Cursos,Você já sentiu desafios para entrar no mercado de trabalho devido ao seu gênero,Você tem interesse em empreender?,Qual é a sua situação diante ao mercado de trabalho?,"Você usa redes sociais (exemplo: X, Facebook, Instagram, Youtube)",Com quais finalidades você usaria o Aplicativo Incluses
0,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Não,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo,Sim,Prefiro cursos online,Sim,Não,Empregado (a),Sim,Todos acima
3,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo,Tenho interesse,"Não tenho preferência, pode ser online ou pres...",Sim,Não,Empregado (a),Sim,Não usaria o app
4,Sim,Menos de 18 anos,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo,Não,"Não tenho preferência, pode ser online ou pres...",Não,Talvez,Empregado (a),Sim,Fazer cursos de qualificação


In [3]:
display(df.describe())

,Você participa da comunidade LGBTQIA+,Idade,Gênero,Orientação Sexual,Localização,Nível de Escolaridade,Você usa aplicativos para procurar oportunidades de emprego?,Preferência por Cursos,Você já sentiu desafios para entrar no mercado de trabalho devido ao seu gênero,Você tem interesse em empreender?,Qual é a sua situação diante ao mercado de trabalho?,"Você usa redes sociais (exemplo: X, Facebook, Instagram, Youtube)",Com quais finalidades você usaria o Aplicativo Incluses
count,162,47,47,47,47,47,47,47,47,47,47,47,47
unique,2,6,7,7,5,7,3,5,2,3,2,2,5
top,Não,Menos de 18 anos,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo,Sim,"Não tenho preferência, pode ser on-line ou pre...",Não,Talvez,Empregado (a),Sim,Todos acima
freq,115,29,20,16,33,16,23,20,29,19,26,44,16


In [4]:
df = df[df['Você participa da comunidade LGBTQIA+'] == 'Sim']
df['Com quais finalidades você usaria o Aplicativo Incluses'] = df['Com quais finalidades você usaria o Aplicativo Incluses'].replace({"Encontrar oportunidades de emprego": "sim", "Fazer cursos de qualificação": "sim", "Divulgar conteúdos profissionais": "sim",'Divulgar conteúdos profissionais\xa0':"sim" ,"Todos acima": "sim", "Não usaria o app": "não"})

df['Idade'] = df['Idade'].replace({"Menos de 18 anos": "jovem", "18-24 anos": "jovem", "25-34 anos": "adulto",'35-44 anos':"adulto" ,"45-54 anos": "adulto", "55 anos ou mais": "idoso"})


display(df.head(5))
display(df.describe())

,Você participa da comunidade LGBTQIA+,Idade,Gênero,Orientação Sexual,Localização,Nível de Escolaridade,Você usa aplicativos para procurar oportunidades de emprego?,Preferência por Cursos,Você já sentiu desafios para entrar no mercado de trabalho devido ao seu gênero,Você tem interesse em empreender?,Qual é a sua situação diante ao mercado de trabalho?,"Você usa redes sociais (exemplo: X, Facebook, Instagram, Youtube)",Com quais finalidades você usaria o Aplicativo Incluses
2,Sim,jovem,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo,Sim,Prefiro cursos online,Sim,Não,Empregado (a),Sim,sim
3,Sim,jovem,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo,Tenho interesse,"Não tenho preferência, pode ser online ou pres...",Sim,Não,Empregado (a),Sim,não
4,Sim,jovem,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo,Não,"Não tenho preferência, pode ser online ou pres...",Não,Talvez,Empregado (a),Sim,sim
5,Sim,jovem,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo,Não,"Não tenho preferência, pode ser online ou pres...",Sim,Não,Empregado (a),Sim,não
14,Sim,jovem,Homem transgênero,Prefiro não informar,Região Sudeste,Nenhum,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Sim,Desempregado (a),Sim,sim


,Você participa da comunidade LGBTQIA+,Idade,Gênero,Orientação Sexual,Localização,Nível de Escolaridade,Você usa aplicativos para procurar oportunidades de emprego?,Preferência por Cursos,Você já sentiu desafios para entrar no mercado de trabalho devido ao seu gênero,Você tem interesse em empreender?,Qual é a sua situação diante ao mercado de trabalho?,"Você usa redes sociais (exemplo: X, Facebook, Instagram, Youtube)",Com quais finalidades você usaria o Aplicativo Incluses
count,47,47,47,47,47,47,47,47,47,47,47,47,47
unique,1,3,7,7,5,7,3,5,2,3,2,2,2
top,Sim,jovem,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo,Sim,"Não tenho preferência, pode ser on-line ou pre...",Não,Talvez,Empregado (a),Sim,sim
freq,47,31,20,16,33,16,23,20,29,19,26,44,41


In [5]:
df_resposta = df['Com quais finalidades você usaria o Aplicativo Incluses']
df_atributo = df.iloc[:, :-1]

#Resposta
label_encoder = LabelEncoder()

#Treino
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

In [6]:
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())
df_resposta = pd.DataFrame(label_encoder.fit_transform(df_resposta))


smote = SMOTE(random_state=12,k_neighbors=1)
df_atributo, df_resposta = smote.fit_resample(df_atributo, df_resposta)

In [7]:
df_atributos_treino, df_atributos_teste, df_resposta_treino, df_resposta_teste = train_test_split(df_atributo, df_resposta, test_size=0.25, random_state=12)

atributos_treino = df_atributos_treino.values
resposta_treino = df_resposta_treino.values

atributos_teste = df_atributos_teste.values
resposta_teste = df_resposta_teste.values




# # Aplicar SMOTE no conjunto de treino, ajustando o número de vizinhos para evitar o erro
# smote = SMOTE(random_state=12, k_neighbors=1)
# atributos_treino, resposta_treino = smote.fit_resample(df_atributos_treino, df_resposta_treino)

# # Convertendo para arrays
# atributos_teste = df_atributos_teste.values
# resposta_teste = df_resposta_teste.values

In [8]:
classificador_bayes = GaussianNB()
modelo = classificador_bayes.fit(df_atributos_treino, df_resposta_treino)

probabilidades = classificador_bayes.predict_proba(atributos_teste)
classificacao = classificador_bayes.predict(atributos_teste)

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [9]:
# print("Probabilidades das classes para os dados de teste:\n", probabilidades)
# print("Classificação dos dados de teste:\n", classificacao)

taxa_de_acerto = accuracy_score(resposta_teste, classificacao)
print("Taxa de acerto", taxa_de_acerto)
print("\nRelatório de Classificação:\n", classification_report(resposta_teste, classificacao))
print("Matriz de Confusão:\n", confusion_matrix(resposta_teste, classificacao))

Taxa de acerto 0.6190476190476191

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.75      0.30      0.43        10
           1       0.59      0.91      0.71        11

    accuracy                           0.62        21
   macro avg       0.67      0.60      0.57        21
weighted avg       0.67      0.62      0.58        21

Matriz de Confusão:
 [[ 3  7]
 [ 1 10]]


In [10]:
numfolds = 5
gnb = GaussianNB()

kf = KFold(n_splits=numfolds, shuffle=True, random_state=12)

validaçãoCruzada = cross_val_score(gnb, df_atributo, df_resposta, cv=kf)
media = validaçãoCruzada.mean()

print(validaçãoCruzada)
print(media)

[0.64705882 0.82352941 0.8125     0.875      1.        ]
0.8316176470588236


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-pack

In [11]:
predictions = cross_val_predict(gnb, df_atributo, df_resposta, cv=kf)

fold_indices = np.arange(len(predictions)) % numfolds

conf_matrices = [
    confusion_matrix(df_resposta[fold_indices == i], predictions[fold_indices == i])
    for i in range(numfolds)
]

acuracia_matrices = [
    accuracy_score(df_resposta[fold_indices == i], predictions[fold_indices == i])
    for i in range(numfolds)
]

for i, conf_matrix in enumerate(conf_matrices):
    print(f"Fold {i + 1}:\nAcurácia: {round(acuracia_matrices[i]*100)}%\n{conf_matrix}\n")

mean_conf_matrix = np.mean(conf_matrices, axis=0)
mean_acuracia_matrix = np.mean(acuracia_matrices, axis=0)
print("matriz de confusao média:")
print(f"{mean_acuracia_matrix*100:.2f}%")
print(mean_conf_matrix)

Fold 1:
Acurácia: 100%
[[ 7  0]
 [ 0 10]]

Fold 2:
Acurácia: 71%
[[6 3]
 [2 6]]

Fold 3:
Acurácia: 94%
[[7 0]
 [1 8]]

Fold 4:
Acurácia: 88%
[[9 2]
 [0 5]]

Fold 5:
Acurácia: 62%
[[4 3]
 [3 6]]

matriz de confusao média:
82.87%
[[6.6 1.6]
 [1.2 7. ]]


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-pack